In [ ]:
import os
import sys

REPO_DIR = "propaganda-detector"

if not os.path.exists(REPO_DIR):
    print("📥 Cloning repository...")
    !git clone https://github.com/Hannysia/propaganda-detector.git
else:
    print("🔄 Updating repository...")
    %cd {REPO_DIR}
    !git pull
    %cd ..

if os.getcwd().split('/')[-1] != REPO_DIR:
    %cd {REPO_DIR}

print(f"📂 Current directory: {os.getcwd()}")

print("📦 Installing dependencies...")

!pip install -r requirements.txt -q

sys.path.append(os.getcwd())

import wandb
from kaggle_secrets import UserSecretsClient
try:
    user_secrets = UserSecretsClient()
    wandb_key = user_secrets.get_secret("WANDB_API_KEY")
    wandb.login(key=wandb_key)
    print("✅ W&B Logged in")
except:
    print("⚠️ W&B Key not found in Secrets")

print("🔍 Searching for data...")
found_path = None
for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        if file.endswith(".task2-TC.labels"):
            found_path = root
            break
    if found_path: break

if found_path:
    print(f"✅ Data found at: {found_path}")
    
    try:
        from src.utils import build_dataset
        
        print("🚀 1. Starting data parsing...")
        df = build_dataset(found_path)
        
        output_file = "/kaggle/working/dataset.csv"
        df.to_csv(output_file, index=False)
        print(f"✅ Dataset ready: {output_file}")
        
        print("\n🚀 2. Starting training (5 epochs)...")
        !python -m src.train
        
    except ImportError as e:
        print(f"❌ Import error: {e}. Check src/utils/ folder structure.")
    except Exception as e:
        print(f"❌ Execution error: {e}")
else:
    print("❌ Data not found! Please check the Input directory.")